# sPyTial For Data Structures

In [23]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

# Test the improved provider system with various Python objects
from collections import namedtuple
from spytial.provider_system import CnDDataInstanceBuilder
from spytial import diagram

# First, a generic object

In [24]:
# Example data with nested structure
demo_data = {
    "users": [
        {"name": "Alice", "age": 30, "skills": ["Python", "JavaScript"]},
        {"name": "Bob", "age": 25, "skills": ["Java", "C++"]}
    ],
    "projects": {
        "web_app": {"lead": "Alice", "status": "active"},
        "mobile_app": {"lead": "Bob", "status": "planning"}
    },
    "stats": {"total_users": 2, "active_projects": 1}
}


# Debug the inline display
from spytial import diagram

try:
    result = diagram(demo_data, method="inline")
except Exception as e:
    print(f"Error in diagram function: {e}")
    import traceback
    traceback.print_exc()



# Simple Constraints: Binary Trees

You can annotate classes with sPyTial constraints.

- We start with a binary tree, here's the def, here's what you can do for example.
- Then we will add some constraints.
- Then, we will add some directives, and do RB trees?
- Now what about if constraints can't be satisfied? -- then what?


In [25]:
from spytial.annotations import orientation, attribute, hideAtom

# Define a binary tree node
@orientation(selector='{ x : TreeNode, y : TreeNode | x.left = y}', directions=['below', 'left'])
@orientation(selector='{ x : TreeNode, y : TreeNode | x.right = y}', directions=['below', 'right'])
@attribute(field='value')
@hideAtom(selector='NoneType')
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __repr__(self):
        return f"TreeNode({self.value})"
    
# Create a sample binary tree
root = TreeNode(
    value=10,
    left=TreeNode(
        value=5,
        left=TreeNode(3),
        right=TreeNode(7)
    ),
    right=TreeNode(
        value=15,
        left=TreeNode(12),
        right=TreeNode(18)
    )
)

# Visualize the binary tree
try:
    diagram(root, method="inline")
except Exception as e:
    print(f"Error visualizing binary tree: {e}")

# More Complex Data Structure?
Now we will model a more complex data structure, with grouping and rings. Maybe like a ring-protocol or something?

But first, Sets are a great chance to show grouping. 

# Sets


In [26]:
numbers = set([1,22,3,4,55,6,7,8])
diagram(numbers, method="inline")

## But this doesn't quite give us the really nice diagram we want.

In [27]:

## But this doesn't quite give us the sense of ''set''  we want.


# So lets group on the `contains` arrows.
from spytial import group
numbers2 = group(field='contains', groupOn=0, addToGroup=1)(set([1,22,3,4,55,6,7,8]))


diagram(numbers2, method="inline")


# Red Black Tree

Now we extend the previous tree data structure to make Red black trees, with two additional directives for color.

In [ ]:
from spytial.annotations import atomColor

# Define a Red-Black Tree node by extending TreeNode
## TODO: Something isn't workign here in terms of inheritance?
@atomColor(selector='{ x : TreeNode | x.color = red }', value='red')
@atomColor(selector='{ x : TreeNode | x.color = black }', value='black')

class RBTreeNode(TreeNode):
    def __init__(self, value, color, left=None, right=None):
        super().__init__(value, left, right)
        self.color = color

    def __repr__(self):
        return f"RBTreeNode({self.value}, {self.color})"

# Create a sample Red-Black Tree
rb_root = RBTreeNode(
    value=10, color="black",
    left=RBTreeNode(
        value=5, color="red",
        left=RBTreeNode(3, "black"),
        right=RBTreeNode(7, "black")
    ),
    right=RBTreeNode(
        value=15, color="red",
        left=RBTreeNode(12, "black"),
        right=RBTreeNode(18, "black")
    )
)

# Visualize the Red-Black Tree
try:
    diagram(rb_root, method="inline")
except Exception as e:
    print(f"Error visualizing Red-Black Tree: {e}")